<a href="https://colab.research.google.com/github/urmzd/rabbit-classifier/blob/main/src/rc_e1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p /tmp
!curl https://raw.githubusercontent.com/urmzd/rabbit-classifier/main/resources/data.csv --output /tmp/data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9455  100  9455    0     0  47275      0 --:--:-- --:--:-- --:--:-- 47039


In [ ]:
import pandas as pd
import requests
import os
import re
import pathlib

DATA_PATH="/tmp/data.csv"
IMAGE_PATH="/tmp/images"

df = pd.read_csv(DATA_PATH, usecols=range(2))

def init_image_directory(image_path=IMAGE_PATH) -> None:
  os.mkdir(image_path)

def download_image_to_location(link: str) -> pathlib.Path:
  # view  : https://regex101.com/r/3bhDMM/1
  # delete: https://regex101.com/delete/N5sItwbrPF73ZllTnRDltxZ1
  file_name_regex = re.compile(r".*\/(.*(\.(jpeg|jpg|png))?)\??.*", flags=re.IGNORECASE)
  regex_matches = file_name_regex.match(link)

  if not regex_matches:
    raise Exception(f"Failed to match file_name for link {link}")

  print(regex_matches.groups())
  if len(regex_matches.groups()) < 3:
    file_name = regex_matches.group(1) + ".png"
  else:
    file_name = regex_matches.group(1)

  image_request_headers={
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
  }

  image = requests.get(link, headers=image_request_headers)
  print(image.status_code)

  content_path = pathlib.Path(IMAGE_PATH)
  content_path.mkdir(parents=True, exist_ok=True)

  file_path = content_path / file_name

  with open(file_path, "wb") as handle:
    handle.write(image.content)

  return file_path

df

,label,link
0,rabbit,https://upload.wikimedia.org/wikipedia/commons...
1,rabbit,https://upload.wikimedia.org/wikipedia/commons...
2,rabbit,https://www.massaudubon.org/var/ezdemo_site/st...
3,rabbit,https://www.welcomewildlife.com/wp-content/upl...
4,rabbit,https://i.natgeofe.com/k/58df97a7-5c47-44b8-97...
...,...,...
81,hare,https://videohive.img.customer.envatouserconte...
82,hare,https://i.ytimg.com/vi/8hEKqUG-WVc/maxresdefau...
83,hare,https://www.furfeatherandfin.com/media/wysiwyg...
84,hare,https://www.wildlifetrusts.org/sites/default/f...


In [46]:
import cv2
from typing import NewType, Optional
import numpy as np
from google.colab.patches import cv2_imshow

Image = NewType('Image', np.array)

def get_image(file_path: pathlib.Path, show=True) -> Optional[Image]:
  img = cv2.imread(str(file_path))
  if show:
    cv2_imshow(img)
  return img

for index, row in df.iterrows():
  label, link = row
  # if index % 10 == 0:
  path = download_image_to_location(link)  
  get_image(path, show=False)

('Eastern_Cottontail.JPG', None, None)
200
('Wild_rabbit_us.jpg', None, None)
200
('situations-solutions.jpg', None, None)
200
('Cottontail-%C2%A9-James-Chen-Shutterstock-scaled-e1640111841484.jpg', None, None)
200
('full-body-cottontail_4x3.jpg', None, None)
200
('Sylvilagus_audubonii_2.jpg', None, None)
200
('cottontail-rabbits.jpg', None, None)
200
('Cottontail-rabbit-Shutterstock.jpg.aspx', None, None)
200
('latest?cb=20191223185526', None, None)
200
('cottontail.jpg', None, None)
200
('cottontail.jpg', None, None)
200
('03nature-facebookJumbo.jpg?year=2017&h=335&w=640&s=11620681800fe1a11ec9fba675bb413acb65fd5182c0e7059d63268e9b3812d6&k=ZQJBKqZ0VN', None, None)
200
('easterncottontailusfwlewisGorman.jpg', None, None)
200
('NewEnglandCottontail.jpg', None, None)
200
('Eastern_Cottontail_MinniethePookie.jpg', None, None)
200
('eastern-cottontail-rabbit_2x3.jpg', None, None)
200
('Eastern-Cottontail.jpg', None, None)
200
('IMG_7029.jpg', None, None)
200
('Cottontail-rabbit-Donald-Hine